In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.mplot3d.proj3d import proj_transform
from matplotlib.patches import FancyArrowPatch
from curve import Curve, curvePlot
import quadrature4
import quadrature_minimal
from copy import deepcopy
import json
DTYPE="float64"
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{bm}" + "\n" + r"\usepackage{amsmath}"
%matplotlib ipympl

In [ ]:
points = []
resolution = 40
for i in range(resolution):
    theta = 2 * np.pi / resolution * i
    points.append(np.array([np.cos(theta), np.sin(2*theta), 0.2 * np.sin(theta)], dtype=DTYPE))
figure8 = Curve(points)
fig = plt.figure()
ax = fig.add_subplot(projection="3d")
curvePlot(figure8, ax, title="$yolo$")


In [ ]:
# Arrows in 3D
class Arrow3D(FancyArrowPatch):
    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs) 
    
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)

setattr(Axes3D, "arrow3D", _arrow3D)

In [ ]:
# Derivative
ALPHA=2
BETA=4
curve = figure8
ARROWSCALE=0.3
fig = plt.figure(dpi=300)
ax = fig.add_subplot(projection="3d")
ax.view_init(elev=27, azim=-72)
curvePlot(curve, ax)

# Colors
cmap = cm.Wistia
l2Gradient = quadrature4.dEnergy(curve, ALPHA, BETA)
l2GradientNorms = np.array([np.linalg.norm(l2Gradient[i]) for i in range(len(l2Gradient))])
maxGradient = max(l2GradientNorms)
l2GradientNorms = l2GradientNorms / maxGradient
l2Gradient = l2Gradient * (-1/maxGradient)
cmapNormalizer = Normalize(vmin=0, vmax=1)

# Plot the vectors
for i in range(len(l2Gradient)):
    ax.arrow3D(curve[i][0], curve[i][1], curve[i][2], l2Gradient[i][0], l2Gradient[i][1], l2Gradient[i][2],
               mutation_scale=3,
               color=cmap(cmapNormalizer(l2GradientNorms[i])))

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_zlim(-1.5, 1.5)
fig.colorbar(cm.ScalarMappable(norm=cmapNormalizer, cmap=cmap), ax=ax, orientation="horizontal", label=r"Relative Magnitude of $-\text{Grad}_{L^2} \boldsymbol{\Gamma}^k$")
plt.show()

#fig.savefig("../L2Derivative.png", transparent=True)
